# STEP 5a: Apply Best Selected Model on the Master Test Dataset - Data pre-processing and NLP transformation
> * Processing numercial and categorical features (as per Step 2)
* Processing and transforming text feature Snippet according to NLP methodology defined in Step 3
* Consolidating data and saving dataset for modeling in a csv file
* **Warning**: It requires to download and save the reference file for the Word2vec processing in order to perform tasks in this Notebook
* Link to the dataset source: https://archive.ics.uci.edu/ml/datasets/News+Popularity+in+Multiple+Social+Media+Platforms#

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.fftpack as sp

%matplotlib inline
import matplotlib.pyplot as plt
from adjustText import adjust_text

import re
from sklearn.preprocessing import Normalizer
import os
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer
from sklearn.naive_bayes import ComplementNB, MultinomialNB

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, GaussianNB

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.datasets import make_circles
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
import scikitplot as skplt

# t_NSE dimensionality reduction
from sklearn.manifold import TSNE

# NLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import gensim
import gensim.parsing.preprocessing as gsp
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

import random
from sklearn import ensemble

from sklearn.model_selection import StratifiedShuffleSplit

import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# Activate Seaborn style
sns.set()

## Load the file

In [2]:
# Importing the file and creating a dataframe
df_master = pd.read_csv(
    "Master_test_final.csv", low_memory=False, skipinitialspace=True
)

In [3]:
# display all columns
pd.set_option("display.max_columns", None)

In [4]:
# remove the Unnamed column
df_master.drop("Unnamed: 0", axis=1, inplace=True)
df_master.shape

(20000, 26)

In [5]:
df_master.head()

,Post_ID,Date,Month,Company,Snippet,TW_Hashtags,Type_Channel,Country,Country 2,Sentiment,ALL_Author,ALL_Impact,ALL_Backlinks,ALL_Thread_Entry_Type,ALL_Tot_Monthly_Vis,ALL_MozRank,TW_Account_Type,TW_Account_Name,TW_NbImpressions,TW_KredInfluence,TW_KredOutreach,TWFB_NbReach,TW_NbFollowers,TW_NbFollowing,TW_Account_Retweet_of,TW_NbTweets
0,883320256,2017-04-28,April,Thomson Reuters,legal education affect market tmsnrt,No TW hashtag,twitter,United Kingdom,United Kingdom,neutral,TRLegalUKI,63,49850734,post,6.000000e+09,9.6,organisational,tw account not identified,12204.0,621.0,0.0,1348,1530.0,825.0,not a rt,4124.0
1,747534914,2017-01-05,January,Thomson Reuters,slovenia toughens prevent another influx migra...,No TW hashtag,twitter,Argentina,Argentina,neutral,INFOS_EN,53,49850734,post,6.000000e+09,9.6,organisational,infos_en,5772.0,798.0,7.0,798,5772.0,81.0,not a rt,387162.0
2,210103078,2017-04-02,April,Thomson Reuters,zambia power short zambia launches switch perc...,No TW hashtag,twitter,United States,United States,neutral,BrlttanyDuguay,28,49850734,share,6.000000e+09,9.6,Not identified,tw account not identified,44.0,410.0,0.0,410,44.0,2239.0,anthoniaorji,563.0
3,908485890,2017-01-26,January,Thomson Reuters,congrats deserved award winner gwgia twitter a...,gwgia,twitter,United States,United States,positive,WomensLeadrshp,28,49850734,post,6.000000e+09,9.6,organisational,tw account not identified,50.0,417.0,0.0,417,50.0,63.0,not a rt,190.0
4,701086584,2017-05-22,May,Thomson Reuters,dozen attorneys general defend subsidy payment...,"insurance, obamacare",twitter,United States,United States,neutral,StephOliverTR5,21,49850734,post,6.000000e+09,9.6,individual,tw account not identified,10.0,325.0,0.0,325,10.0,81.0,not a rt,209.0


## Cleaning the dataframe based on Step 2 tasks (numerical and categorical features)

In [6]:
# remove the following columns: ALL_Backlinks, ALL_MozRank, ALL_Tot_Monthly_Vis, TW_KredInfluence, TW_NbImpressions, Type_Channel, Month
df_master.drop(
    [
        "ALL_Backlinks",
        "ALL_MozRank",
        "ALL_Tot_Monthly_Vis",
        "TW_KredInfluence",
        "TW_NbImpressions",
        "Type_Channel",
        "Month",
    ],
    axis=1,
    inplace=True,
)
df_master.shape

(20000, 19)

### Numerical features

In [7]:
# select the integer columns
df_num = df_master.select_dtypes(include=[np.int64, np.float64])
# df_num.drop('Post_ID', axis=1, inplace=True)

In [8]:
df_num.shape

(20000, 7)

In [9]:
# Add a Log1p variables to the DataFrame
df_num["Log_TW_KredOutreach"] = np.log1p(df_num["TW_KredOutreach"])
df_num["Log_Nbreach"] = np.log1p(df_num["TWFB_NbReach"])
df_num["Log_TW_NbFollowers"] = np.log1p(df_num["TW_NbFollowers"])
df_num["Log_TW_NbFollowing"] = np.log1p(df_num["TW_NbFollowing"])
df_num["Log_TW_NbTweets"] = np.log1p(df_num["TW_NbTweets"])
df_num.head()

,Post_ID,ALL_Impact,TW_KredOutreach,TWFB_NbReach,TW_NbFollowers,TW_NbFollowing,TW_NbTweets,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets
0,883320256,63,0.0,1348,1530.0,825.0,4124.0,0.000000,7.207119,7.333676,6.716595,8.324821
1,747534914,53,7.0,798,5772.0,81.0,387162.0,2.079442,6.683361,8.660947,4.406719,12.866601
2,210103078,28,0.0,410,44.0,2239.0,563.0,0.000000,6.018593,3.806662,7.714231,6.335054
3,908485890,28,0.0,417,50.0,63.0,190.0,0.000000,6.035481,3.931826,4.158883,5.252273
4,701086584,21,0.0,325,10.0,81.0,209.0,0.000000,5.786897,2.397895,4.406719,5.347108


In [10]:
# Add a Log1p variables to the DataFrame
df_num["Log_TW_KredOutreach"] = np.log1p(df_num["TW_KredOutreach"])
df_num["Log_Nbreach"] = np.log1p(df_num["TWFB_NbReach"])
df_num["Log_TW_NbFollowers"] = np.log1p(df_num["TW_NbFollowers"])
df_num["Log_TW_NbFollowing"] = np.log1p(df_num["TW_NbFollowing"])
df_num["Log_TW_NbTweets"] = np.log1p(df_num["TW_NbTweets"])
df_num.head()

,Post_ID,ALL_Impact,TW_KredOutreach,TWFB_NbReach,TW_NbFollowers,TW_NbFollowing,TW_NbTweets,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets
0,883320256,63,0.0,1348,1530.0,825.0,4124.0,0.000000,7.207119,7.333676,6.716595,8.324821
1,747534914,53,7.0,798,5772.0,81.0,387162.0,2.079442,6.683361,8.660947,4.406719,12.866601
2,210103078,28,0.0,410,44.0,2239.0,563.0,0.000000,6.018593,3.806662,7.714231,6.335054
3,908485890,28,0.0,417,50.0,63.0,190.0,0.000000,6.035481,3.931826,4.158883,5.252273
4,701086584,21,0.0,325,10.0,81.0,209.0,0.000000,5.786897,2.397895,4.406719,5.347108


### Categorical features

In [11]:
# Create a dataframe with categorical features
df_cat = df_master[
    [
        "Post_ID",
        "Company",
        "Country 2",
        "Sentiment",
        "ALL_Thread_Entry_Type",
        "TW_Account_Type",
    ]
]

#### One-Hot encoding of following features: Company, Country 2, ALL_Thread_Entry_Type, TW_Account_type

In [12]:
df_cat_nom = df_cat[
    ["Company", "Country 2", "ALL_Thread_Entry_Type", "TW_Account_Type"]
]

In [13]:
encode_norm = pd.get_dummies(df_cat_nom)

In [14]:
encode_norm.shape

(20000, 38)

In [15]:
# Transform dtype from uint8 to int64
encode_norm = encode_norm.astype(np.int64)

#### Encode ordinal feature: sentiment

In [16]:
df_cat = df_cat.replace({"Sentiment": {"negative": -1, "neutral": 0, "positive": 1}})

In [17]:
df_sentiment_encod = df_cat[["Sentiment"]]

### Concatenate different datasets created for feature engineering in one dataset

In [18]:
# Merge encoded nominal and ordinal features
encode_tot = pd.merge(
    df_sentiment_encod, encode_norm, right_index=True, left_index=True
)
encode_tot.shape

(20000, 39)

In [19]:
# Merge encoded features with log_transform features
df_feateng = pd.merge(encode_tot, df_num, right_index=True, left_index=True)

In [20]:
df_feateng.shape

(20000, 51)

In [21]:
# Select features to be added from the master file (twitter_eda)
select_feat = df_master[
    ["Date", "Snippet", "TW_Hashtags", "ALL_Author", "TW_Account_Name"]
]

### Masterdata_raw: Reference dataset with all numerical and categorical features

In [22]:
# Merge dataset with all feautre engineered and the missing features from the original master file
masterdata_raw2 = pd.merge(df_feateng, select_feat, right_index=True, left_index=True)

In [23]:
masterdata_raw2.shape

(20000, 56)

### Masterdata_ml2: Dataframe with selected features eligible for the modeling phase

In [24]:
masterdata_ml2 = masterdata_raw2.drop(
    [
        "TW_KredOutreach",
        "TWFB_NbReach",
        "TW_NbFollowers",
        "TW_NbFollowing",
        "TW_NbTweets",
        "Date",
    ],
    axis=1,
)

In [25]:
masterdata_ml2.shape

(20000, 50)

In [26]:
masterdata_ml2.head()

,Sentiment,Company_Clarivate,Company_Informa,Company_Pearson,Company_RELX Group,Company_Thomson Reuters,Company_Wolters Kluwer,Country 2_Argentina,Country 2_Australia,Country 2_Belgium,Country 2_Brazil,Country 2_Canada,Country 2_Ecuador,Country 2_France,Country 2_Germany,Country 2_Hong Kong,Country 2_India,Country 2_Italy,Country 2_Japan,Country 2_Mexico,Country 2_Netherlands,Country 2_Other,Country 2_Philippines,Country 2_Russia,Country 2_Serbia,Country 2_Singapore,Country 2_South Africa,Country 2_Spain,Country 2_Switzerland,Country 2_United Arab Emirates,Country 2_United Kingdom,Country 2_United States,Country 2_Venezuela,ALL_Thread_Entry_Type_post,ALL_Thread_Entry_Type_reply,ALL_Thread_Entry_Type_share,TW_Account_Type_Not identified,TW_Account_Type_individual,TW_Account_Type_organisational,Post_ID,ALL_Impact,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets,Snippet,TW_Hashtags,ALL_Author,TW_Account_Name
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,883320256,63,0.000000,7.207119,7.333676,6.716595,8.324821,legal education affect market tmsnrt,No TW hashtag,TRLegalUKI,tw account not identified
1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,747534914,53,2.079442,6.683361,8.660947,4.406719,12.866601,slovenia toughens prevent another influx migra...,No TW hashtag,INFOS_EN,infos_en
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,210103078,28,0.000000,6.018593,3.806662,7.714231,6.335054,zambia power short zambia launches switch perc...,No TW hashtag,BrlttanyDuguay,tw account not identified
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,908485890,28,0.000000,6.035481,3.931826,4.158883,5.252273,congrats deserved award winner gwgia twitter a...,gwgia,WomensLeadrshp,tw account not identified
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,701086584,21,0.000000,5.786897,2.397895,4.406719,5.347108,dozen attorneys general defend subsidy payment...,"insurance, obamacare",StephOliverTR5,tw account not identified


### Save Masterdata_mlto a csv file

In [27]:
# masterdata_ml.to_csv('masterdata_ml.csv')

## Preparing text feature based on Step 3 procedure

### Create a dataframe with selected text feature used for modeling

In [28]:
snippets = masterdata_ml2["Snippet"].astype(str)

In [29]:
snippets.head()

0                 legal education affect market tmsnrt
1    slovenia toughens prevent another influx migra...
2    zambia power short zambia launches switch perc...
3    congrats deserved award winner gwgia twitter a...
4    dozen attorneys general defend subsidy payment...
Name: Snippet, dtype: object

### Snippet pre-processing

In [30]:
# Create a list of strings, where each string is an article title
snippet_list = [snippet for snippet in snippets]

big_list = " ".join(snippet_list)

# Tokenize the string into words
tokens = word_tokenize(big_list)

# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]

words = [word for word in words if not word in stop_words]

# Print first 10 words
words[:10]

['legal',
 'education',
 'affect',
 'market',
 'tmsnrt',
 'slovenia',
 'toughens',
 'prevent',
 'another',
 'influx']

### Load Word2Vec model trained on the large corpus: GoogleNews (using Gensim library)
* Library documentation: https://radimrehurek.com/gensim/models/keyedvectors.html
* Link to the dataset source: https://archive.ics.uci.edu/ml/datasets/News+Popularity+in+Multiple+Social+Media+Platforms#

In [31]:
# Load Word2Vec model
model = gensim.models.KeyedVectors.load_word2vec_format(
    "C:/Users/fbaff/Documents/Data Science/Dataset/GoogleNews-vectors-negative300.bin",
    binary=True,
)

### Perform snippet embeding via averaging word embeding for each of them

In [32]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)


# Pre-processing to keep each snippet as a doc (not individual words)
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()]
    return doc


# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)


# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes
    a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus) if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [33]:
# Preprocess the corpus
corpus = [preprocess(title) for title in snippet_list]

# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(
    corpus, snippet_list, lambda doc: has_vector_representation(model, doc)
)

# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, snippet_list, lambda doc: (len(doc) != 0))

82 docs removed
0 docs removed


In [34]:
# Initialize an array for the size of the corpus
x = []
for doc in corpus:  # append the vector for each document
    x.append(document_vector(model, doc))

X = np.array(x)  # list to array

In [35]:
# Create a dataframe with full document embedding and the dataframe resulting from the EDA process
df_all_vectors = pd.DataFrame(x)

df_all_vectors["Title"] = titles_list

main_all_vectors2 = pd.concat((df_all_vectors, masterdata_ml2), axis=1)

# Get rid of vectors that couldn't be matched with the original dataframe
main_all_vectors2.dropna(axis=0, inplace=True)

main_all_vectors2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Title,Sentiment,Company_Clarivate,Company_Informa,Company_Pearson,Company_RELX Group,Company_Thomson Reuters,Company_Wolters Kluwer,Country 2_Argentina,Country 2_Australia,Country 2_Belgium,Country 2_Brazil,Country 2_Canada,Country 2_Ecuador,Country 2_France,Country 2_Germany,Country 2_Hong Kong,Country 2_India,Country 2_Italy,Country 2_Japan,Country 2_Mexico,Country 2_Netherlands,Country 2_Other,Country 2_Philippines,Country 2_Russia,Country 2_Serbia,Country 2_Singapore,Country 2_South Africa,Country 2_Spain,Country 2_Switzerland,Country 2_United Arab Emirates,Country 2_United Kingdom,Country 2_United States,Country 2_Venezuela,ALL_Thread_Entry_Type_post,ALL_Thread_Entry_Type_reply,ALL_Thread_Entry_Type_share,TW_Account_Type_Not identified,TW_Account_Type_individual,TW_Account_Type_organisational,Post_ID,ALL_Impact,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets,Snippet,TW_Hashtags,ALL_Author,TW_Account_Name
0,-0.138428,-0.001816,-0.094147,-0.067780,0.025970,0.033813,0.153992,0.030151,0.143555,-0.002014,-0.060566,-0.019653,-0.105133,0.110291,-0.145081,0.074402,0.076069,0.164062,0.085945,-0.117432,-0.030182,0.036865,0.077515,0.047623,-0.047241,-0.006287,-0.124878,0.029999,-0.014572,0.034546,0.074783,-0.128784,0.033325,-0.019424,-0.014988,-0.050194,0.038574,0.126709,0.039429,-0.017273,-0.025116,0.051147,0.066839,0.020996,-0.204346,-0.104460,0.114651,0.086975,0.012146,0.053604,0.069885,-0.046417,-0.036331,0.003418,0.001617,0.047729,-0.138580,-0.008850,0.042419,-0.139648,-0.076782,-0.106018,0.070251,0.004883,-0.063820,-0.006165,-0.002777,0.138550,-0.021240,-0.050659,-0.080811,-0.001167,0.229248,0.077187,-0.018524,-0.099266,0.133667,0.090820,-0.006561,0.005432,-0.046799,-0.053297,-0.085510,0.131714,-0.066681,0.065979,-0.113937,0.167267,-0.072144,-0.023148,0.364502,-0.130798,0.001923,-0.012207,0.022400,-0.132309,0.091675,-0.052185,0.148911,0.051208,0.023056,0.094238,-0.066589,0.051221,-0.082222,-0.156311,-0.048859,0.025757,0.098572,-0.231934,0.003052,0.028931,-0.167236,0.023560,0.130112,-0.031830,0.011963,-0.057617,0.189087,0.021606,-0.033417,0.081055,-0.078444,-0.006195,-0.038208,-0.072754,-0.167480,-0.033936,-0.050157,0.010376,0.132202,-0.013321,-0.171143,-0.048309,0.035980,-0.074005,0.004211,0.035095,0.057941,0.011475,-0.026855,-0.011841,-0.084412,0.116455,-0.064774,-0.078735,0.200439,0.014816,-0.117486,-0.029541,0.147957,0.024048,-0.075333,-0.042768,-0.109776,-0.041073,-0.059814,-0.212830,-0.122742,-0.102210,0.015442,-0.016171,0.127441,0.037109,0.099136,0.062531,0.070763,-0.082703,0.135780,-0.108429,-0.156799,-0.030884,0.082245,-0.073799,0.073853,-0.093262,0.139313,0.018860,-0.042843,0.139038,-0.199860,-0.138550,-0.013519,-0.049561,0.004272,-0.010231,0.006226,0.125061,0.044098,0.145905,-0.013123,-0.033630,-0.064331,0.030533,-0.118195,0.023830,-0.118866,-0.120972,0.137676,-0.098999,-0.089844,0.010254,0.087921,-0.083313,-0

In [36]:
main_all_vectors2.shape

(19848, 351)

#### Transform 'All Impact' variable in 3 bins as defined in the Step 4
> * In order to preserve the integrity of Master Test variables and stay coherent with the modeling approach, we will perform a bin transformation with the same bin width as defined in the Step 4
* Bins range are: [ 0., 30., 41., 80.]
* Remove not relevant features and save the dataframe to a csv file

In [37]:
main_all_vectors2["All_impact_bins"] = pd.cut(
    x=main_all_vectors2["ALL_Impact"], bins=[0, 30, 41, 80], labels=[0, 1, 2]
)

In [38]:
main_all_vectors2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Title,Sentiment,Company_Clarivate,Company_Informa,Company_Pearson,Company_RELX Group,Company_Thomson Reuters,Company_Wolters Kluwer,Country 2_Argentina,Country 2_Australia,Country 2_Belgium,Country 2_Brazil,Country 2_Canada,Country 2_Ecuador,Country 2_France,Country 2_Germany,Country 2_Hong Kong,Country 2_India,Country 2_Italy,Country 2_Japan,Country 2_Mexico,Country 2_Netherlands,Country 2_Other,Country 2_Philippines,Country 2_Russia,Country 2_Serbia,Country 2_Singapore,Country 2_South Africa,Country 2_Spain,Country 2_Switzerland,Country 2_United Arab Emirates,Country 2_United Kingdom,Country 2_United States,Country 2_Venezuela,ALL_Thread_Entry_Type_post,ALL_Thread_Entry_Type_reply,ALL_Thread_Entry_Type_share,TW_Account_Type_Not identified,TW_Account_Type_individual,TW_Account_Type_organisational,Post_ID,ALL_Impact,Log_TW_KredOutreach,Log_Nbreach,Log_TW_NbFollowers,Log_TW_NbFollowing,Log_TW_NbTweets,Snippet,TW_Hashtags,ALL_Author,TW_Account_Name,All_impact_bins
0,-0.138428,-0.001816,-0.094147,-0.067780,0.025970,0.033813,0.153992,0.030151,0.143555,-0.002014,-0.060566,-0.019653,-0.105133,0.110291,-0.145081,0.074402,0.076069,0.164062,0.085945,-0.117432,-0.030182,0.036865,0.077515,0.047623,-0.047241,-0.006287,-0.124878,0.029999,-0.014572,0.034546,0.074783,-0.128784,0.033325,-0.019424,-0.014988,-0.050194,0.038574,0.126709,0.039429,-0.017273,-0.025116,0.051147,0.066839,0.020996,-0.204346,-0.104460,0.114651,0.086975,0.012146,0.053604,0.069885,-0.046417,-0.036331,0.003418,0.001617,0.047729,-0.138580,-0.008850,0.042419,-0.139648,-0.076782,-0.106018,0.070251,0.004883,-0.063820,-0.006165,-0.002777,0.138550,-0.021240,-0.050659,-0.080811,-0.001167,0.229248,0.077187,-0.018524,-0.099266,0.133667,0.090820,-0.006561,0.005432,-0.046799,-0.053297,-0.085510,0.131714,-0.066681,0.065979,-0.113937,0.167267,-0.072144,-0.023148,0.364502,-0.130798,0.001923,-0.012207,0.022400,-0.132309,0.091675,-0.052185,0.148911,0.051208,0.023056,0.094238,-0.066589,0.051221,-0.082222,-0.156311,-0.048859,0.025757,0.098572,-0.231934,0.003052,0.028931,-0.167236,0.023560,0.130112,-0.031830,0.011963,-0.057617,0.189087,0.021606,-0.033417,0.081055,-0.078444,-0.006195,-0.038208,-0.072754,-0.167480,-0.033936,-0.050157,0.010376,0.132202,-0.013321,-0.171143,-0.048309,0.035980,-0.074005,0.004211,0.035095,0.057941,0.011475,-0.026855,-0.011841,-0.084412,0.116455,-0.064774,-0.078735,0.200439,0.014816,-0.117486,-0.029541,0.147957,0.024048,-0.075333,-0.042768,-0.109776,-0.041073,-0.059814,-0.212830,-0.122742,-0.102210,0.015442,-0.016171,0.127441,0.037109,0.099136,0.062531,0.070763,-0.082703,0.135780,-0.108429,-0.156799,-0.030884,0.082245,-0.073799,0.073853,-0.093262,0.139313,0.018860,-0.042843,0.139038,-0.199860,-0.138550,-0.013519,-0.049561,0.004272,-0.010231,0.006226,0.125061,0.044098,0.145905,-0.013123,-0.033630,-0.064331,0.030533,-0.118195,0.023830,-0.118866,-0.120972,0.137676,-0.098999,-0.089844,0.010254,0.087

In [39]:
# Check the bin values
main_all_vectors2["All_impact_bins"].unique()

[2, 0, 1, NaN]
Categories (3, int64): [0 < 1 < 2]

In [40]:
# Check the number of NaN after transformation
main_all_vectors2.isnull().sum()  # 2 data points have not been assigned to classes. We will remove them as it will not impact the modeling

0                  0
1                  0
2                  0
3                  0
4                  0
                  ..
Snippet            0
TW_Hashtags        0
ALL_Author         0
TW_Account_Name    0
All_impact_bins    1
Length: 352, dtype: int64

In [41]:
df_modeling = main_all_vectors2.dropna(subset=["All_impact_bins"])

In [42]:
df_modeling.shape

(19847, 352)

In [43]:
# Check the distribution of data points
df_modeling[
    "All_impact_bins"
].value_counts()  # Split of data is almost equally distributed as the principle taken in the previous modeling phases

0    6872
1    6682
2    6293
Name: All_impact_bins, dtype: int64

In [44]:
# Remove not relevant features to match the final
final_modeling = df_modeling.drop(
    [
        "Post_ID",
        "ALL_Impact",
        "Snippet",
        "TW_Hashtags",
        "ALL_Author",
        "TW_Account_Name",
        "Title",
    ],
    axis=1,
)

In [45]:
final_modeling.shape

(19847, 345)

In [46]:
# Create a csv file
# final_modeling.to_csv('Final_modeling.csv')